# EDA


- The os module has a perfect method to list files in a directory.
- Pandas json normalize could work here but is not necessary to convert the JSON data to a dataframe.
- You may need a nested for-loop to access each sale!
- We've put a lot of time into creating the structure of this repository, and it's a good example for future projects.  In the file functions_variables.py, there is an example function that you can import and use.  If you have any variables, functions or classes that you want to make, they can be put in the functions_variables.py file and imported into a notebook.  Note that only .py files can be imported into a notebook. If you want to import everything from a .py file, you can use the following:
```python
from functions_variables import *
```
If you just import functions_variables, then each object from the file will need to be prepended with "functions_variables"\
Using this .py file will keep your notebooks very organized and make it easier to reuse code between notebooks.

In [28]:
# (this is not an exhaustive list of libraries)
import pandas as pd
import numpy as np
import os
import json
from pprint import pprint

## Data Importing

In [29]:
# load one file first to see what type of data you're dealing with and what attributes it has
# see 'all_headings' file 

In [30]:
# loop over all files and put them into a dataframe
from functions_importing import get_data


In [31]:
filepath = r"C:\Users\mebar\Documents\LHL-DataSci\Projects\midterm_project_housing_prices\Data"

In [32]:
raw_data = get_data(filepath)
raw_data.head()

,tags,status,list_date,open_houses,descrip_year_built,descrip_baths_3qtr,descrip_sold_date,descrip_sold_price,descrip_baths_full,descrip_name,...,flags_is_foreclosure,flags_is_plan,flags_is_coming_soon,flags_is_new_listing,community,products,listing_id,price_reduced_amount,location_state,location_city
0,"[carport, community_outdoor_space, cul_de_sac,...",sold,2023-06-29T21:16:25.000000Z,None,1963.0,NaN,2023-09-18,NaN,2.0,None,...,None,None,None,False,None,{'brand_name': 'basic_opt_in'},2957241843,45000.0,Alaska,Juneau
1,None,sold,None,None,NaN,NaN,2023-08-22,NaN,NaN,None,...,None,None,None,None,None,None,None,NaN,Alaska,Juneau
2,None,sold,None,None,NaN,NaN,2023-08-22,NaN,NaN,None,...,None,None,None,None,None,None,None,NaN,Alaska,Juneau
3,None,sold,None,None,NaN,NaN,2023-08-21,NaN,NaN,None,...,None,None,None,None,None,None,None,NaN,Alaska,Juneau
4,None,sold,None,None,NaN,NaN,2023-08-21,NaN,NaN,None,...,None,None,None,None,None,None,None,NaN,Alaska,Juneau


## Data Cleaning and Wrangling

At this point, ensure that you have all sales in a dataframe.
- Take a quick look at your data (i.e. `.info()`, `.describe()`) - what do you see?
- Is each cell one value, or do some cells have lists?
- What are the data types of each column?
- Some sales may not actually include the sale price (target).  These rows should be dropped.
- There are a lot of NA/None values.  Should these be dropped or replaced with something?
    - You can drop rows or use various methods to fills NA's - use your best judgement for each column 
    - i.e. for some columns (like Garage), NA probably just means no Garage, so 0
- Drop columns that aren't needed
    - Don't keep the list price because it will be too close to the sale price. Assume we want to predict the price of houses not yet listed

In [33]:
from functions_importing import extract_tags

In [34]:
data = extract_tags(raw_data)

In [35]:
data.shape

(8159, 194)

Empty columns: 

In [36]:
data.loc[:,['descrip_name','descrip_baths_1qtr','flags_is_new_construction','flags_is_for_rent','flags_is_subdivision','flags_is_contingent',
            'flags_is_pending','flags_is_plan','flags_is_coming_soon']].isna().value_counts()

descrip_name  descrip_baths_1qtr  flags_is_new_construction  flags_is_for_rent  flags_is_subdivision  flags_is_contingent  flags_is_pending  flags_is_plan  flags_is_coming_soon
True          True                True                       True               True                  True                 True              True           True                    8159
Name: count, dtype: int64

In [37]:
# drop empty columns, old tags column
data = data.drop(['descrip_name','descrip_baths_1qtr','flags_is_new_construction','flags_is_for_rent','flags_is_subdivision','flags_is_contingent',
            'flags_is_pending','flags_is_plan','flags_is_coming_soon','tags'],axis=1)

In [38]:
# fill expanded tags columns' NaNs with 0s
data.iloc[:,0:155] = data.iloc[:,0:155].fillna(0)

Garage tags are cumulative; anything with a garage_3_or_more tag will also have a garage_1_or_more and garage_2_or_more. These seem to be refering to the total car capacity, as the descrip_garage column can be more than 3, but if the garage_3_or_more tag is present it will be at **least** 3. Some properties with garage_n_or_more tags have blank descrip_garage values, so that should be filled in and used as the only garage column.

In [39]:
display(data.loc[[0,6,244,2136],['garage_1_or_more','garage_2_or_more','garage_3_or_more','descrip_garage']])
print('only descrip filled in:',len(data[(data['garage_1_or_more']!=1.0)&(data['garage_2_or_more']!=1.0)&(data['garage_3_or_more']!=1.0)&(data['descrip_garage']>0)]))
print('missing descrip:',len(data[(data['descrip_garage'].isna())&((data['garage_1_or_more']>0)|(data['garage_2_or_more']>0)|(data['garage_3_or_more']>0))]))
print('inside the rows missing descrip:')
display(data[((data['garage_1_or_more']>0)|(data['garage_2_or_more']>0)|(data['garage_3_or_more']>0))&(data['descrip_garage'].isna())][['garage_1_or_more','garage_2_or_more','garage_3_or_more']].sum())

,garage_1_or_more,garage_2_or_more,garage_3_or_more,descrip_garage
0,1.0,0.0,0.0,1.0
6,1.0,1.0,0.0,2.0
244,1.0,1.0,1.0,3.0
2136,1.0,1.0,1.0,6.0


only descrip filled in: 38
missing descrip: 339
inside the rows missing descrip:


garage_1_or_more    339.0
garage_2_or_more      0.0
garage_3_or_more      0.0
dtype: float64

Since all the rows with an empty descrip_garage value only have garage_1 tags, fill descip_garage with 1 for those rows. 

In [40]:
data.loc[((data[(data['descrip_garage']==0.0)&(data['garage_1_or_more']>0)].index).tolist()),'descrip_garage'] = 1

# descrip_garage was not part of the mass tags NaN fill, so do that now
data.loc[:,'descrip_garage'] = data['descrip_garage'].fillna(0)

In [41]:
# drop the now redundant columns, rename the updated one
data = data.drop(['garage_1_or_more','garage_2_or_more','garage_3_or_more'],axis=1)
data = data.rename(columns={'descrip_garage': 'garage_car_spaces'})

According to Google, forced air and central heat are both types of heating, just with slightly different mechanisms, so those can be combined into one column 'heating' and then dropped.

In [42]:
print('properties with either forced air or central heat:')
print(len(data[(data['forced_air']>0) | (data['central_heat']>0)]))

properties with either forced air or central heat:
4594


In [43]:
# create heating column, default value 0
data['heating'] = 0.0

# assign the rows with those tags
data.loc[((data[(data['heating']==0.0)&((data['central_heat']>0)|(data['forced_air']>0))].index).tolist()),'heating'] = 1.0

In [44]:
#drop old columns
data = data.drop(['central_heat','forced_air'],axis=1)

solar_panels is completely redundant with solar_system and never occurs on its own; renaming solar_system 'solar_power' and dropping solar_panel. 

In [49]:
print('solar system or panels:',len(data[(data['solar_system']>0) | (data['solar_panels']>0)]))
print('both:',len(data[(data['solar_system']>0) & (data['solar_panels']>0)]))
print('only panels:',len(data[(data['solar_system']==0) & (data['solar_panels']>0)]))

solar system or panels: 151
both: 84
only panels: 0


In [50]:
data = data.rename(columns={'solar_system': 'solar_power'})
data = data.drop('solar_panels',axis=1)

### Dealing with Tags

Consider the fact that with tags, there are a lot of categorical variables.
- How many columns would we have if we OHE tags, city and state?
- Perhaps we can get rid of tags that have a low frequency.

In [45]:
# OHE categorical variables/ tags here
# tags will have to be done manually

### Dealing with Cities

In [54]:
data.iloc[:,-3:]

,location_state,location_city,heating
0,Alaska,Juneau,0.0
1,Alaska,Juneau,0.0
2,Alaska,Juneau,0.0
3,Alaska,Juneau,0.0
4,Alaska,Juneau,0.0
...,...,...,...
8154,West Virginia,Charleston,1.0
8155,West Virginia,Charleston,0.0
8156,West Virginia,Charleston,0.0
8157,West Virginia,Charleston,0.0


- Sales will vary drastically between cities and states.  Is there a way to keep information about which city it is without OHE?
- Could we label encode or ordinal encode?  Yes, but this may have undesirable effects, giving nominal data ordinal values.
- What we can do is use our training data to encode the mean sale price by city as a feature (a.k.a. Target Encoding)
    - We can do this as long as we ONLY use the training data - we're using the available data to give us a 'starting guess' of the price for each city, without needing to encode city explicitly
- If you replace cities or states with numerical values (like the mean price), make sure that the data is split so that we don't leak data into the training selection. This is a great time to train test split. Compute on the training data, and join these values to the test data
- Note that you *may* have cities in the test set that are not in the training set. You don't want these to be NA, so maybe you can fill them with the overall mean

In [46]:
# perform train test split here
# do something with state and city

## Extra Data - STRETCH

> This doesn't need to be part of your Minimum Viable Product (MVP). We recommend you write a functional, basic pipeline first, then circle back and join new data if you have time

> If you do this, try to write your downstream steps in a way it will still work on a dataframe with different features!

- You're not limited to just using the data provided to you. Think/ do some research about other features that might be useful to predict housing prices. 
- Can you import and join this data? Make sure you do any necessary preprocessing and make sure it is joined correctly.
- Example suggestion: could mortgage interest rates in the year of the listing affect the price? 

In [47]:
# import, join and preprocess new data here

## EDA/ Visualization

Remember all of the EDA that you've been learning about?  Now is a perfect time for it!
- Look at distributions of numerical variables to see the shape of the data and detect outliers.    
    - Consider transforming very skewed variables
- Scatterplots of a numerical variable and the target go a long way to show correlations.
- A heatmap will help detect highly correlated features, and we don't want these.
    - You may have too many features to do this, in which case you can simply compute the most correlated feature-pairs and list them
- Is there any overlap in any of the features? (redundant information, like number of this or that room...)

In [48]:
# perform EDA here

## Scaling and Finishing Up

Now is a great time to scale the data and save it once it's preprocessed.
- You can save it in your data folder, but you may want to make a new `processed/` subfolder to keep it organized